<a href="https://colab.research.google.com/github/defort0970/Jupyter/blob/master/bookscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install --upgrade woocommerce
!pip install googletrans==3.1.0a0
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
from google.colab import drive
import sys
drive.mount('/content/drive/')
sys.path.append('/content/drive/MyDrive/ColabNotebooks/')

from jcmodule import *
from const import *

wcapi=getjcapi(JCAPIURL,JCCK1,JCCS1,JCTMO180)

SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/ColabNotebooks/emtional-healingart-61b6632f5288.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, scope) # Your json file here
gc = gspread.authorize(credentials)
sheet = gc.open("左西購物網產品上架規格").sheet1

data = sheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
max_rows = len(sheet.get_all_values())

Requirement already up-to-date: woocommerce in /usr/local/lib/python3.7/dist-packages (3.0.0)
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
from bs4 import BeautifulSoup
import requests
import time
import os
import urllib.request

def finddetail(words,reptext):
    df = []
    for item in words:
        item = item.text.replace(' ','')
        df.append(item)
    df = pd.DataFrame(df)
    newstr = df[df[0].str.contains(reptext)]
    if newstr.empty:
        return ""
    else:
        nwdf = str(newstr).replace('\n','').strip()
    return ''.join(nwdf)[4:].strip()

In [15]:
def fetchbooks(indno):
    idx = indno-2
    url = df.loc[idx][2].split('?')[0]
    urlforimg = url.split('/')[-1]
    html = requests.get(url).text
    soup = BeautifulSoup(html, "lxml")
    images = soup.findAll('img')
    catalogno = df.loc[idx][5]

    bookprice = soup.find('div', { 'class': 'cnt_prod002 clearfix' }).text.split('\n')
    bookinfo = soup.find('div', { 'class': 'type02_p003 clearfix' }).text.split('\n')
    bkki = []
    bkki = bookinfo[-3].replace(' ','')\
    +bookinfo[-2].replace('原文作者','\n原文作者').replace('新功能介紹','').replace('譯者','\n譯者').replace('出版社：','\n出版社：').replace(' ','')\
    +bookinfo[-1].replace('新功能介紹','').replace('語言','\n語言').replace('出版日期','\n出版日期')

    bkprice = bookprice[4].replace("定價：","").replace("原價：","").replace("元","")
    sheet.update_acell('M'+str(indno), bkprice)
    sheet.update_acell('N'+str(indno), bkki)


    if len(soup.find_all('div', { 'class': 'mod type02_p002 clearfix' }))>0:#書籍類別
        for ps in soup.find_all('div', { 'class': 'mod type02_p002 clearfix' }):
            tile = ps.select('h1')[0].text.split('\n')
            cctile = tile[0].replace(':','').replace('?','')
            slugglong = [sub for sub in jtranslate(cctile,'en') if sub.isalpha() or sub.isspace()]
            slugglong = ''.join(slugglong)
            slugg ='-'.join(slugglong.split()[:6])
            slugg =slugg[:16].replace(':','').replace('?','').replace('--','')
    elif len(soup.find_all('div', { 'class': 'cnt_prod002 clearfix' }))>0:#桌遊類別
        for ps in soup.find_all('div', { 'class': 'mod prd001' }):
            tile = ps.select('h1')[0].text.split('\n')
            cctile = tile[0].replace(':','').replace('?','')
            slugglong = [sub for sub in jtranslate(cctile,'en') if sub.isalpha() or sub.isspace()]
            slugglong = ''.join(slugglong)
            slugg ='-'.join(slugglong.split()[:6])
            slugg =slugg[:16].replace(':','').replace('?','').replace('--','')

    sheet.update_acell('V'+str(indno),str(slugg))#Slug 英文名稱
    coverimg = "https://www.books.com.tw/img/"+str(urlforimg[:3])+'/'+str(urlforimg[3:6])+'/'+str(urlforimg[6:8])+'/'+str(urlforimg)+".jpg"
    sheet.update_acell('W'+str(indno),coverimg)#coverimg

    ix = []
    ixt = []

    without_duplicates = []
    for image in images:
        imgurl = image['src']
        if imgurl[:70] == "https://im2.book.com.tw/image/getImage?i=https://www.books.com.tw/img/" or\
            imgurl[:70] == "https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/" :
            if imgurl[81:91] == urlforimg:#[10]:
                a1image = imgurl[:91]+".jpg&w=500&h=600"
                andpostion = imgurl.find('&')
                ix = imgurl[41:andpostion]
                ixt.extend([ix])
                [without_duplicates.append(element) for element in ixt if element not in without_duplicates]
            nodimages = ','.join(without_duplicates[1:])

    for detailinfo in soup.find_all('div', { 'class': 'mod_b type02_m058 clearfix' }):
        checkitem =['ISBN：','叢書系列：','規格：','出版地：','本書分類：']
        infolist = [finddetail(detailinfo.select('li'),checkitem[i]).replace(checkitem[i],'') for i in range(5)]
        sheet.update_acell('L'+str(indno),infolist[0])#ISBN Number
        sheet.update_acell('O'+str(indno),infolist[1])#叢書系列
        sheet.update_acell('P'+str(indno),infolist[2])#規格
        sheet.update_acell('Q'+str(indno),infolist[3])#出版地
        sheet.update_acell('R'+str(indno),infolist[4].replace(' ',''))#分類
        dimsn = str(infolist[2]).replace('cm','').split('/')[2] #新版規格
        isbnno = infolist[0]
        try:
            length = dimsn.split('x')[0]
            width = dimsn.split('x')[1]
            if len(dimsn.split('x'))==2:
                height = ''
            else:
                height = dimsn.split('x')[2]
        except IndexError as e:
            length = ""
            width = ""
            height = ""

    sheet.update_acell('S'+str(indno),length)#尺寸：長
    sheet.update_acell('T'+str(indno),width)#尺寸：寬
    sheet.update_acell('U'+str(indno),height)#尺寸：高
  
    #封面圖先上傳 GD 並且轉換尺寸，拋轉到 Cloud Storage，再轉入 Woocommerce
    drfilepath = "/content/drive/MyDrive/JCBimages/"
    imgrzpath = str(isbnno)+slugg+"-rzcvr"+".jpg"
    firstfilename = drfilepath+str(isbnno)+slugg+".jpg"
    cvfilename = drfilepath+imgrzpath

    try:
        urllib.request.urlretrieve(coverimg[:50]+".jpg",firstfilename)
        imgresizefun(firstfilename,cvfilename,JCCOVER_H,JCCOVER_W)
    except urllib.error.HTTPError as e:
        pass

    upload_to_bucket(imgrzpath,cvfilename,"staging.emtional-healingart.appspot.com")
    cvig = "https://storage.googleapis.com/staging.emtional-healingart.appspot.com/"+imgrzpath
    allimages = cvig+','+nodimages
    print(allimages)

    accordion = ['[/accordion-item]','[accordion-item title="作者介紹"]','[accordion-item title="目錄"]','[accordion-item title="序言"]','[accordion-item title="內容連載"]']
    descontanct =[]
    for ct,i in zip(soup.find_all('div', { 'class': 'mod_b type02_m057 clearfix' }),range(4)):
        ct = ct.select('div')[0]#.replace('\xa0','').replace('\n\n','').replace('\u3000','').replace('\r','')#.split('\n')
        if i==0:
            descontanct.append(ct)
        elif i == 1:
            descontanct.append(accordion[1])
            descontanct.append(ct)
            descontanct.append(accordion[0])
        elif i == 2:
            descontanct.append(accordion[2])
            descontanct.append(ct)
            descontanct.append(accordion[0])
        elif i == 3:
            descontanct.append(accordion[3])
            descontanct.append(ct)
            descontanct.append(accordion[0])
    # print(descontanct)
    descontanct.append(accordion[4])
    for ctseon in soup.find_all('div', { 'class': 'mod_b type02_m059 clearfix' }):
        ctseon = ctseon.select('div')[0]
        descontanct.append(ctseon)

    urlcontant = url.split('/')
    urlbook = "https://www.books.com.tw/web/sys_serialtext/?item="+str(urlcontant[-1])#str(urlcontant[10])
    for i in range(2,12):
        urlpage = urlbook+"&page="+str(i)
        urlgo = requests.get(urlpage).text
        soup = BeautifulSoup(urlgo, "lxml")
        time.sleep(0.1*i)
        if soup:
            for cttn in soup.find_all('div', { 'class': 'mod type02_specific_02 clearfix' }):
                cttn = cttn.select('div')[1]
                descontanct.append(cttn)
    descontanct.append(accordion[0])
      
    toremove = [
        '<div>','</div>','<div class="type02_gradient" style="display:none;">',\
        '<div class="cont">','<div style="text-align: center">','<div class="bd">',\
        '</div><div class="type02_gradient" style="display:none;',\
        '<div class="content" style="height:auto;">','"></div>','</div>',\
        '</div><div class="type02_gradient" id="M201105_0_getProdTextInfo_P00a400020009_g3" style="display:block;','</div>]',\
        '</div>,','</div><div class="type02_gradient" id="M201105_0_getProdTextInfo_P00a400020009_g2" style="display:block;">',\
        '<div class="content more_off" id="M201105_0_getProdTextInfo_P00a400020009_h3">',\
        '<div class="content more_off" id="M201105_0_getProdTextInfo_P00a400020009_h2">','<div style="text-align: center;">',\
        '<div class="type02_gradient" id="M201105_0_getProdTextInfo_P00a400020009_g2" style="display:block;">',\
        '<div style="text-align: right;">','<div class="type02_gradient" id="M201105_0_getProdTextInfo_P00a400020009_g3" style="display:block;">',\
        ', ','<div class="content">','<div class="type02_gradient" style="display:block;','<div class="hr_shadow clearfix">',\
        '<b class="bl"></b>','　　']

    for tm in toremove:
        descontanct = str(descontanct).replace(tm,"")
        descontanct = str(descontanct).replace("'","")
        descontanct= descontanct[1:-1]
        sheet.update_acell('AA'+str(indno),descontanct)#內文
        
    dfnew = allimages.split(',')
    image_data = []
    dfnew = allimages.split(',')
    for k in range(len(dfnew)):
        if dfnew[k]:
            image_data.append({ "src": dfnew[k],
                                "position": k,
                                "name": str(cctile[:10])+'-'+str(k),
                                "alt": str(slugg)+'-'+str(k),})
        
    data = {"name": str(cctile),
            "slug": str(slugg),
            "type": "simple",
            "sku": str(isbnno),
            "short_description": str(bkki),
            "description":str(descontanct),
            "regular_price":str(bkprice),
            "categories": [{'id': str(catalogno)}],
            "manage_stock": True,
            "stock_quantity": 0,
            "stock_status": "outofstock",
            "weight": "",
            "dimensions": { "length":str(length),
                            "width": str(width),
                            "height": str(height)},
            "images": image_data}
    try:
        wcapi.post("products/", data).json()
    except JSONDecodeError as e:
        print('Fail in POST Jason')
        pass

    print("Finished!!")


In [16]:
def jctages():
    Flag = True
    page =1 
    data = []
    dftags = pd.DataFrame()
    while Flag:
        tagdata = wcapi.get("products/tags/?page="+str(page)).json()
        data = pd.json_normalize(tagdata)
        if len(tagdata) == 0: # no more tags
            Flag = False
            break
        page += 1
        dftags = dftags.append(data)
    
    dftags = dftags.reset_index(drop=True)
    return dftags

In [17]:
def alltagsupdate(dfbuy,dbtags,dbdesn,dbsku,wcapi):
    dftags = jctages()
    dbtags = dbtags.fillna("")
    dftags['name'] = dftags['name'].astype(str)

    for i in range(len(dbtags)):
        if not dbtags.loc[i]:
            dbtags.loc[i] =""
        else:
            dbtags.loc[i] = ','.join(dbtags.loc[i].split('|'))

    dbdesn = dbdesn.astype(str)
    temp = []
    for i in range(len(dbtags)):
        for k in dbtags.loc[i].split(','):
            for j in range(len(dftags['name'])):
                if k.count(dftags['name'][j]):
                    temp.append(str(dftags['id'].loc[j]))
        dbtags.loc[i] = ','.join(temp)
        temp = []

    for i in range(len(dbdesn)):
        for j in range(len(dftags['name'])):
            if dbdesn[i].count(dftags['name'][j]):
                dbtags.loc[i] = dbtags.loc[i]+','+str(dftags['id'].loc[j])
                dbtags.loc[i] = ','.join(list(dict.fromkeys(dbtags.loc[i].split(','))))
        if not len(dbtags[i]):
            dbtags[i] = ","
        if dbtags[i][0] == ',':
            dbtags[i]=dbtags[i][1:]

    for i,ptno,tagname in zip(range(len(dfbuy)),dbsku,dbtags):
        productlist = False
        while not productlist:
            productlist = wcapi.get("products/?sku="+str(ptno)).json()
            time.sleep(22)
            print("wait for updates")

        productid = productlist[0]['id']
        dfnew=dbtags[i].split(',')
        data=[]
        for k in range(len(dfnew)):
            data.append({"id": dfnew[k]},)
        data = {"tags":data,
                "status": "draft",}
        
        try:
            wcapi.put("products/"+str(productid), data).json()
            print(productid,data)
        except JSONDecodeError as e:
            print('Fail in POST Jason')
            pass

In [18]:
for i in range(int(max_rows),0,-1):
    if sheet.acell('A'+str(i)).value == "TRUE":
        print(sheet.acell('B'+str(i)).value)
        fetchbooks(i)
        ironmen_dict = {"Tag":"","Description": str(sheet.acell('AA'+str(i)).value),"Product SKU": str(sheet.acell('L'+str(i)).value)}
        dfbuy = pd.DataFrame(ironmen_dict,index=[0])
        alltagsupdate(dfbuy,dfbuy['Tag'],dfbuy['Description'],dfbuy['Product SKU'],wcapi)
        print('Tags update OK.')
        sheet.update_acell('A'+str(i),"FALSE")
        sheet.update_acell('H'+str(i),"TRUE")
        time.sleep(0.2)
    else:
        time.sleep(i/100)
        print('Loop:',i)
        pass

print('作業完成，圓滿結束！')

海奧華預言：第九級星球的九日旅程‧奇幻不思議的真實見聞
1.285 26 0 26 800 986
https://storage.googleapis.com/staging.emtional-healingart.appspot.com/9789869854825Theoward-Prophec-rzcvr.jpg,https://www.books.com.tw/img/001/084/60/0010846057_bc_01.jpg
Finished!!
wait for updates
330043 {'tags': [{'id': '1140'}, {'id': '1100'}, {'id': '1081'}, {'id': '1230'}, {'id': '1025'}, {'id': '1187'}, {'id': '1237'}, {'id': '1199'}, {'id': '1227'}, {'id': '1203'}, {'id': '1235'}, {'id': '1229'}, {'id': '1190'}, {'id': '1101'}, {'id': '1234'}, {'id': '1051'}, {'id': '1244'}, {'id': '891'}, {'id': '1209'}, {'id': '1145'}], 'status': 'draft'}
Tags update OK.
Loop: 8
Loop: 7
Loop: 6
Loop: 5
Loop: 4
Loop: 3
Loop: 2
Loop: 1
作業完成，圓滿結束！
